In [1]:
import Pkg;
using SpecialFunctions
using Plots; plotly();
using Ipopt
using JuMP

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\Noah Klausner\.julia\packages\Plots\SjqWU\src\backends.jl:372


In [2]:
p_gac = 2.1*10^6
p_soil = 1.3*10^6
vol = 2*1.82*1.22
K_A = 0.0016
P_A = 20
K_B = 0.0032
P_B = 40
K_C = 0.0048
P_C = 80
lim = (600*10^-9)*(1.3*10^6)

80

In [3]:
function C(Co,x,v,t,D,f_gac,f_A,f_B,f_C)
    m_soil = p_soil*vol*(1-f_gac)
    m_gac = p_gac*vol*f_gac
    k_gac = K_A*f_A + K_B*f_B + K_C*f_C
    k_d = k_gac*f_gac
    p_d = (m_soil+m_gac)/vol
    theta = 1-(((m_gac/p_gac)+(m_soil/p_soil))/vol)
    R = 1 + (k_d*p_d)/theta
    C = Co*erfc((x-(v/R)*t)/(2*sqrt((D/R)*t)))
    return C
end

C (generic function with 1 method)

In [4]:
function Cost_rate(Co,x,v,t,D,f_gac,f_A,f_B,f_C)
    costperfilter = p_gac*f_gac*vol*(f_A*P_A + f_B*P_B + f_C*P_C)
    T = 0
    Conc = C(Co,x,v,T,D,f_gac,f_A,f_B,f_C)
    while Conc < lim
        T = T + 0.1
        Conc = C(Co,x,v,T,D,f_gac,f_A,f_B,f_C)
    end
    Cost_rate = costperfilter*T
    return Cost_rate
end     

Cost_rate (generic function with 1 method)

In [11]:
C_model = Model(Ipopt.Optimizer)
@variable(C_model, 0 <= f_gac <= 1)
@variable(C_model, 0 <= f_A <= 1)
@variable(C_model, 0 <= f_B <= 1)
@variable(C_model, 0 <= f_C <= 1)
@objective(C_model, Min, Cost_rate(2.8785,2,0.00000317,8000,0.0001268392,f_gac,f_A,f_B,f_C))
@constraint(C_model, c1, f_A + f_B + f_C = 1)

LoadError: [91mCannot multiply a quadratic expression by an aff. expression[39m

In [12]:
optimize!(C_model)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [13]:
@show value(f_gac)

value(f_gac) = 0.12280690884667908


0.12280690884667908

In [14]:
@show value(f_A)

value(f_A) = 0.12280690884667908


0.12280690884667908

In [15]:
@show value(f_B)

value(f_B) = 0.12280690884667908


0.12280690884667908

In [16]:
@show value(f_C)

value(f_C) = 0.12280690884667908


0.12280690884667908